In [12]:
import h5py 
import tensorflow as tf
import tensorflow.keras.layers as tfl
import numpy as np

print(tf.config.list_physical_devices('GPU'))


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
with h5py.File("../../EIDSeg_Dataset/cache/eidseg_64x64_binary_any.h5", "r") as f:
    X_train_org = f["X_train"][:]
    Y_train_org = f["Y_train"][:]
    X_test_org  = f["X_test"][:]
    Y_test_org  = f["Y_test"][:]

m_train = X_train_org.shape[0]
m_test = X_test_org.shape[0]
num_px =X_train_org.shape[1]

print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Height/Width of each image: num_px = " + str(num_px))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_set_x shape: " + str(X_train_org.shape))
print ("train_set_y shape: " + str(Y_train_org.shape))
print ("test_set_x shape: " + str(X_test_org.shape))
print ("test_set_y shape: " + str(Y_test_org.shape))

Number of training examples: m_train = 2612
Number of testing examples: m_test = 327
Height/Width of each image: num_px = 64
Each image is of size: (64, 64, 3)
train_set_x shape: (2612, 64, 64, 3)
train_set_y shape: (1, 2612)
test_set_x shape: (327, 64, 64, 3)
test_set_y shape: (1, 327)


In [5]:
def building_damage_model():

    model = tf.keras.Sequential([
        # Input + first convolution block
        tf.keras.layers.Conv2D(
            filters=32,
            kernel_size=(3, 3),
            strides=1,
            padding='same',
            activation='relu',
            input_shape=(64, 64, 3)
        ),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2),

        # Second convolution block
        tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=(3, 3),
            strides=1,
            padding='same',
            activation='relu'
        ),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2),

        # Classifier
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    return model


In [6]:
model = building_damage_model()

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


/home/ahmed/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2026-02-01 13:36:46.298419: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2026-02-01 13:36:46.301634: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/A

In [7]:
Y_train = Y_train_org.reshape(-1)
Y_test  = Y_test_org.reshape(-1)


In [8]:
X_train = X_train_org / 255.0
X_test  = X_test_org / 255.0


# data clearly is biased 

In [17]:
print("Train positives:", np.sum(Y_train))
print("Train negatives:", Y_train.shape[0] - np.sum(Y_train))

print("Test positives:", np.sum(Y_test))
print("Test negatives:", Y_test.shape[0] - np.sum(Y_test))

Train positives: 1847
Train negatives: 765
Test positives: 242
Test negatives: 85


In [9]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     1,048,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,068,097 (4.07 MB)

 Trainable params: 1,068,097 (4.07 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
history = model.fit(
    X_train,
    Y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, Y_test)
)


Epoch 1/20


I0000 00:00:1769946738.321930    6981 service.cc:145] XLA service 0x7ef3e0004be0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1769946738.322414    6981 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2026-02-01 13:52:18.366092: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2026-02-01 13:52:18.612360: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


18/82 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7182 - loss: 0.6288

I0000 00:00:1769946741.196615    6981 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


82/82 ━━━━━━━━━━━━━━━━━━━━ 8s 49ms/step - accuracy: 0.7041 - loss: 0.6169 - val_accuracy: 0.7401 - val_loss: 0.5795
Epoch 2/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7071 - loss: 0.6075 - val_accuracy: 0.7401 - val_loss: 0.5730
Epoch 3/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7071 - loss: 0.6065 - val_accuracy: 0.7401 - val_loss: 0.5746
Epoch 4/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7071 - loss: 0.6085 - val_accuracy: 0.7401 - val_loss: 0.5728
Epoch 5/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7071 - loss: 0.6056 - val_accuracy: 0.7401 - val_loss: 0.5789
Epoch 6/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7071 - loss: 0.6076 - val_accuracy: 0.7401 - val_loss: 0.5774
Epoch 7/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7071 - loss: 0.5993 - val_accuracy: 0.7401 - val_loss: 0.5642
Epoch 8/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7071 - loss: 0.5962 - val_accuracy: 0.7401 - val_loss: 0.

In [11]:
test_loss, test_acc = model.evaluate(X_test, Y_test)
print("Test accuracy:", test_acc)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7523 - loss: 0.5300 
Test accuracy: 0.752293586730957


In [13]:
pred_probs = model.predict(X_test)
pred_labels = (pred_probs > 0.5).astype(int).reshape(-1)

11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step


In [14]:
print("Predicted destroyed ratio:", pred_labels.mean())
print("True destroyed ratio:", Y_test.mean())


Predicted destroyed ratio: 0.9877675840978594
True destroyed ratio: 0.7400611620795107


# The model is learning to put all ones, because of the data bias to 1 (destroyed)

In [18]:

num_0 = np.sum(Y_train == 0)
num_1 = np.sum(Y_train == 1)

print(num_0, num_1)


765 1847


In [19]:
total = num_0 + num_1

weight_for_0 = total / (2 * num_0)
weight_for_1 = total / (2 * num_1)

class_weights = {
    0: weight_for_0,
    1: weight_for_1
}

print(class_weights)


{0: 1.70718954248366, 1: 0.7070925825663238}


In [20]:
history_w = model.fit(
    X_train,
    Y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, Y_test),
    class_weight=class_weights
)


Epoch 1/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.5965 - loss: 0.6865 - val_accuracy: 0.7523 - val_loss: 0.5962
Epoch 2/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.6930 - loss: 0.6587 - val_accuracy: 0.6789 - val_loss: 0.6151
Epoch 3/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6788 - loss: 0.6507 - val_accuracy: 0.5260 - val_loss: 0.6590
Epoch 4/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6719 - loss: 0.6440 - val_accuracy: 0.5780 - val_loss: 0.6434
Epoch 5/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6708 - loss: 0.6351 - val_accuracy: 0.6758 - val_loss: 0.5898
Epoch 6/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6826 - loss: 0.6252 - val_accuracy: 0.6789 - val_loss: 0.5762
Epoch 7/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6780 - loss: 0.6241 - val_accuracy: 0.6697 - val_loss: 0.5836
Epoch 8/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6819 - loss: 0.6128 - val_accuracy: 0.6820 - v

In [24]:
test_loss2, test_acc2 = model.evaluate(X_test, Y_test)
print("Test accuracy:", test_acc2)
pred_probs = model.predict(X_test)



11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7554 - loss: 0.5170 
Test accuracy: 0.7553516626358032
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


In [25]:
pred_labels2 = (pred_probs > 0.5).astype(int).reshape(-1)
print("Predicted destroyed ratio:", pred_labels.mean())
print("True destroyed ratio:", Y_test.mean())

Predicted destroyed ratio: 0.7522935779816514
True destroyed ratio: 0.7400611620795107
